In [ ]:
%%bash
pip install matplotlib aiohttp

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# 设置字体为安装的中文字体（根据你的系统配置字体名称）
rcParams['font.sans-serif'] = ['WenQuanYi Zen Hei', 'Noto Sans CJK JP']  # 可根据安装的字体名称修改
rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 测试中文显示
plt.plot([1, 2, 3], [1, 4, 9])
plt.title('中文标题')  # 中文标题
plt.xlabel('时间')  # 中文x轴标签
plt.ylabel('值')  # 中文y轴标签
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 数据
x = np.linspace(0, 10, 100)
y1 = np.sin(x)
y2 = np.cos(x)

# 创建 2 行 1 列的子图
fig, axes = plt.subplots(2, 1, figsize=(8, 6))

# 第一个子图
axes[0].plot(x, y1, label='sin(x)')
axes[0].set_title('Sine Wave')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].legend()

# 第二个子图
axes[1].plot(x, y2, label='cos(x)', color='orange')
axes[1].set_title('Cosine Wave')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].legend()

# 调整布局
plt.tight_layout()

# 显示图形
plt.show()


In [ ]:
import asyncio
import aiohttp
import json
from functools import reduce

def get_stock_url(stock_code):
    base_url = 'https://29.push2.eastmoney.com/api/qt/stock/trends2/sse'
    query_params = {
        "fields1": "f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f17",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58",
        "mpi": "1000",
        "ut": "fa5fd1943c7b386f172d6893dbfba10b",
        "secid": f"{stock_code}",
        "ndays": 1,
        "iscr": 0,
        "iscca": 0,
        "wbp2u": "|0|0|0|web"
        }
    query_string = reduce(lambda x,y:str(x)+'&'+str(y), map(lambda x:str(x)+'='+str(query_params.get(x, '')), query_params))
    url = f"{base_url}?{query_string}"
   
    return url

def get_url_headers(stock_code):
    stock_exchange = {
        "0": "sz",
        "1": "sh"
        }
    ex_key, code = stock_code.split(".")
    headers = {
        'Accept': 'text/event-stream',
        'Accept-Language':  'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Cache-Control':  'no-cache',
        'Connection':  'keep-alive',
        'Origin':  'https://quote.eastmoney.com',
        'Referer':  f'https://quote.eastmoney.com/{stock_exchange[ex_key]}{code}.html',
        'Sec-Fetch-Dest':  'empty',
        'Sec-Fetch-Mode':  'cors',
        'Sec-Fetch-Site':  'same-site',
        'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
        'sec-ch-ua':  '"Not(A:Brand";v="99", "Microsoft Edge";v="133", "Chromium";v="133"',
        'sec-ch-ua-mobile':  '?0',
        'sec-ch-ua-platform':  '"Windows"'
        }
    return headers

def parse_data(stock_code, data):
    data = data[5:]
    data = json.loads(data)
    stock_data = data.get("data", {}) or {}
    
    stock_name = stock_data.get("name")
    trends_datas = stock_data.get("trends")
    if trends_datas:
        # print(trends_datas)
        lastest_trend_data = trends_datas[-1]
        period, open_price, close_price, high_price, \
            low_price, trad_volume, trad_amount, \
                current_price = trends_datas[-1].split(',')
        print()

async def get_stock_data(stock_code):
    url = get_stock_url(code)
    headers = get_url_headers(code)
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    async for line in response.content:
                        if line:
                            message = line.decode('utf-8').strip()
                            parse_data(stock_code, message)

                else:
                    print(f"Failed to connect to {url}, Status code: {response.status}")
    except Exception as e:
        print(f"Error while connecting to {url}: {e}")




"""
中公教育： 0.002607
洛阳钼业： 1.603993
"""
stock_codes = ["0.002607", "1.603993"]

tasks = [get_stock_data(code) for code in stock_codes]
await asyncio.gather(*tasks)



Received from https://29.push2.eastmoney.com/api/qt/stock/trends2/sse?fields1=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f17&fields2=f51,f52,f53,f54,f55,f56,f57,f58&mpi=1000&ut=fa5fd1943c7b386f172d6893dbfba10b&secid=0.002607&ndays=1&iscr=0&iscca=0&wbp2u=|0|0|0|web: data: {"rc":0,"rt":10,"svr":177617710,"lt":1,"full":1,"dlmkts":"","data":{"code":"002607","market":0,"type":6,"status":0,"name":"中公教育","decimal":2,"preSettlement":0.0,"preClose":3.42,"beticks":"33300|34200|54000|34200|41400|46800|54000","trendsTotal":241,"time":1740987291,"kind":1,"prePrice":3.42,"hisPrePrices":[{"date":20250303,"prePrice":3.42}],"trends":["2025-03-03 09:30,3.41,3.41,3.41,3.41,13405,4571105.00,3.410","2025-03-03 09:31,3.42,3.42,3.45,3.42,63171,21658201.00,3.425","2025-03-03 09:32,3.43,3.40,3.43,3.37,92722,31512804.00,3.411","2025-03-03 09:33,3.39,3.40,3.41,3.39,24171,8214194.00,3.409","2025-03-03 09:34,3.40,3.39,3.41,3.39,15033,5110236.00,3.408","2025-03-03 09:35,3.39,3.36,3.40,3.36,44363,14983707.00,3.403",

CancelledError: 